# Experiment Analysis

This notebook loads experiment results and creates publication-quality figures with three different font styles.

**Prerequisites:** Run experiments first with `./scripts/run.sh -n experiment`

**Font configurations demonstrated:**
1. **LaTeX + Computer Modern** - Classic LaTeX look with TeX rendering
2. **Times New Roman** - Journal-style without TeX dependency
3. **Matplotlib defaults** - Standard matplotlib appearance

In [1]:
# Setup: change to project root, imports, and configuration
import os
os.chdir("../")

import glob
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from src import utils

# Configuration
name = "experiment"  # experiment name (must match output files)
width_pt = 469       # document text width in points
sns.set_theme(context='paper', style='ticks', font_scale=1)

# The following command adds to the path the directory containing the local LaTeX installation (here, TinyTeX)
# This should not be necessary, unless you installed LaTeX manually in your home folder
# Try running the notebook without this line first
os.environ['PATH'] = f"{os.path.expanduser('~/.TinyTeX/bin/x86_64-linux')}:{os.environ['PATH']}"


In [2]:
# Load experiment results
files = glob.glob(f"./outputs/{name}__*.json")
print(f"Found {len(files)} result files")

rows = []
for filepath in files:
    with open(filepath, "r") as f:
        data = json.load(f)
    params = {k: v for k, v in data.items() if k != 'seed_results'}
    for seed_result in data['seed_results']:
        row = {**params, **seed_result}
        rows.append(row)

df = pd.DataFrame(rows)
df.head()

Found 4 result files


,exp_name,N,iterations,prob,master_seed,num_seeds,avg_num_abstained,most_common_winner,seed
0,experiment,10,10,0.1,42,0,0.9,1,42
1,experiment,10,10,0.5,42,0,5.2,5,42
2,experiment,20,10,0.1,42,0,1.8,8,42
3,experiment,20,10,0.5,42,0,10.2,18,42


## Plot 1: LaTeX + Computer Modern

Classic LaTeX appearance with Computer Modern fonts and TeX rendering for math.

In [3]:
# LaTeX + Computer Modern (requires LaTeX installation)
utils.latexify()  # defaults: Computer Modern with usetex=True

fig_width, fig_height = utils.get_fig_dim(width_pt, fraction=0.6)
fig, ax = plt.subplots(figsize=(fig_width, fig_height))

# specify color palette
palette = sns.color_palette('husl', 2)

# make main plot
sns.barplot(data=df, x="N", hue="prob", y="avg_num_abstained", errorbar=('ci', 95), palette=palette, ax=ax)
sns.despine(ax=ax)

# polish labels and legends
ax.set_xlabel(r"$N$")
ax.set_ylabel("Average number of\nabstained voters")
ax.legend(title="Probability", loc='upper left')

fig.tight_layout()
fig.savefig(f'figures/{name}_cm.pdf', dpi=300)

## Plot 2: Times New Roman

Journal-style appearance without TeX dependency. Works on any system with Times New Roman installed.

In [4]:
# Times New Roman (no LaTeX required)
utils.latexify(font_serif='Times New Roman', usetex=False)

fig_width, fig_height = utils.get_fig_dim(width_pt, fraction=0.6)
fig, ax = plt.subplots(figsize=(fig_width, fig_height))

# specify color palette
palette = sns.color_palette('husl', 2)

# make main plot
sns.barplot(data=df, x="N", hue="prob", y="avg_num_abstained", errorbar=('ci', 95), palette=palette, ax=ax)
sns.despine(ax=ax)

# polish labels and legends
ax.set_xlabel("N")
ax.set_ylabel("Average number of\nabstained voters")
ax.legend(title="Probability", loc='upper left')

fig.tight_layout()
fig.savefig(f'figures/{name}_times.pdf', dpi=300)

Font 'Times New Roman' not found, using 'Times' instead


## Plot 3: Matplotlib Defaults

Standard matplotlib appearance with default fonts and styling.

In [5]:
# Matplotlib defaults
utils.latexify(use_defaults=True)  # resets to matplotlib defaults

fig_width, fig_height = utils.get_fig_dim(width_pt, fraction=0.6)
fig, ax = plt.subplots(figsize=(fig_width, fig_height))

# specify color palette
palette = sns.color_palette('husl', 2)

# make main plot
sns.barplot(data=df, x="N", hue="prob", y="avg_num_abstained", errorbar=('ci', 95), palette=palette, ax=ax)
sns.despine(ax=ax)

# polish labels and legends
ax.set_xlabel("N")
ax.set_ylabel("Average number of\nabstained voters")
ax.legend(title="Probability", loc='upper left')

fig.tight_layout()
fig.savefig(f'figures/{name}_default.pdf', dpi=300)